# Deep Learning

딥러닝은 모형들의 output이 다시 input으로 들어가서 학습을 이어나가는 Layer 구조를 가지는 모형. 모형의 특징을 결정하는데는 다음과 같은 요소들이 필요



*   모델 Building
    * Connectivity Patterns
    * Nonlinearity Modules
    * Loss function

*   모델 학습
    * Optimization
    * Hyper Parameters

# Connectivity Pattern

딥러닝은 여러 레이어를 쌓아나가는 구조. 뉴런은 각 레이어에 있으며, 레이어들간의 연결관계에 따라서 패턴이 나뉨.

* Fully-Connected
* COnvolutional
* Dilated
* Recurrent
* Skip / Residual
* Random

여기서는 간단한 Connectivity Pattern만 정리





## 1. FC Layer
뉴럴넷의 가장 간단한 아키텍쳐. 필요한 것은 레이어의 갯수와 뉴런의 갯수 그리고, 각 layer들의 연결 패턴을 고려해야함. 이번 케이스에서는 input, hidden, output layer를 각각 하나씩 가지고 있는 모듈을 만듬. connectivity pattern은 fully connected로 함.
* Layer number
* Neuron number
* connectivity pattern

In [24]:
layer_dims = [5, 4, 3, 1]

In [25]:
import numpy as np

def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    cache = Z
    return A, cache

def relu(Z):
    A = np.maximum(0, Z)
    assert(A.shape == Z.shape)
    cache = Z
    return A, cache

In [26]:
def initialize_parameters_deep(layer_dims):
    # dict 객체 생성
    parameters = {}
    # 총 layer들의 길이 계산
    L = len(layer_dims)
    # 레이어들을 돌면서, 레이어들 간의 weight와 bias의 초기값의 난수 생성
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1]) # *0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

        # assert를 통해 dimension 맞춤. 틀릴시 error 발생
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

    return parameters

In [27]:
parameters = initialize_parameters_deep(layer_dims)
parameters

{'W1': array([[-0.48801736,  0.36878669, -0.2937001 ,  0.57356559,  0.25579563],
        [-0.89222788,  0.15173943,  0.37397418,  0.04629986,  0.72582663],
        [-0.01686243, -0.36319177, -0.15185667,  0.63758029,  0.93262974],
        [-0.1446226 ,  0.17641405, -0.2385277 ,  0.46009671,  0.09729178]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.]]),
 'W2': array([[-0.23618477, -0.49271991, -0.55171079, -0.47334635],
        [-0.16752299,  0.35649675, -0.05892275,  0.39333935],
        [-0.429569  ,  0.08679653, -0.40196401, -0.41667964]]),
 'b2': array([[0.],
        [0.],
        [0.]]),
 'W3': array([[-0.26504679,  0.35880478,  0.11373019]]),
 'b3': array([[0.]])}

In [28]:
parameters['W1'].shape # param shape 개같이 구성해놓은듯

(4, 5)

In [29]:
def linear_forward(A, W, b):
    # W에 A를 내적. 그 후 b를 더해줌.
    Z = W.dot(A) + b
    # Z의 shape이 input과 weigth의 shape과 동일한지 체크.
    assert(Z.shape == (W.shape[0], A.shape[1]))
    # 계산단계에서 사용한 값을 cache에 저장
    cache = (A, W, b)
    return Z, cache

def linear_activation_forward(A_prev, W, b, activation):
    # Activation func의 종류에 따라서 값을 나눔.
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)

    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)

    # Shape이 input과 weight와 동일한지 체크.
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    # linear 연산과 activation 연산을 cache에 저장.
    cache = (linear_cache, activation_cache)
    return A, cache

In [30]:
def L_model_forward(X, parameters):
    # cache 들의 list.
    caches = []
    A = X
    # weight와 bias가 저장되어 있기 때문에 //2 를 해줘야 layer의 사이즈가 됨.
    L = len(parameters) // 2  # layer 개수

    # hidden layers relu 통과
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)

    # output layer는 sigmoid를 통과하게 함.
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = "sigmoid")
    caches.append(cache)
    assert(AL.shape == (1, X.shape[1]))
    return AL, caches

In [31]:
X = np.random.randn(5, 4)
Y = np.array([[0, 1, 1, 0]])
AL, caches = L_model_forward(X, parameters)

In [32]:
AL

array([[0.55217926, 0.55484082, 0.5       , 0.56343183]])

## 4. Cost Function

우리는 신경망을 통과한 $\hat{y}$값을 찾을 수 있었음. 하지만 우리의 실제 y 레이블과는 다른 값일 가능성이 매우 크기 때문에 이를 반영하여 학습을 시켜야함. Cost function은 여러가지 종류가 있지만, 이번의 경우에는 cross-entropy 함수를 사용하려고함. 이후에 Cost function에 대해서도 정리할 예정.

$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right))$$

### Arguments

AL : 뉴럴넷을 통과해서 나오게된 $\hat{y}$. shape (1, # of examples)<br/>
Y : 실제 "label" vector. shape (1, # of examples)

### Returns

cost : cross-entropy cost

In [33]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = (-1.0/m)*np.sum(np.multiply(Y, np.log(AL)) + np.multiply(1-Y, np.log(1-AL)))
    cost = np.squeeze(cost)
    assert(cost.shape == ())

    return cost

In [34]:
cost = compute_cost(AL, Y)
print("cost = " + str(cost))

cost = 0.7285985463399304


### Backpropagation Process
Backpropagation은 다음과 같은 process를 가지게 됨.
* Linear backward
* Linear -> Activation backward
* Layer -> Layer backward

## Linear backward

Linear 한 영역에서 backward 과정은 다음과 같은 인자를 받게됨.

### Arguments
dZ : Z의 변화량. linear 부분에서 output이 cost function에 대한 gradient를 나타냄.<br/>
cache : forward과정에서 필요한 값을 받아옴. tuple 형태의 (A_prev, W, b) 값들을 받아옴.

### Returns
dA_prev : Linear 구간의 input으로 들어왔었던, 지난 레이어의 activation을 통과한 A가 cost func에 대한 변화량.<br/>
dW : Linear 구간의 weight의 cost func에 대한 변화량.<br/>
db : Linear 구간의 bias의 cost func에 대한 변화량

## Linear-Activation backward

Activation func g(.)에 대해서 Linear-activate backward는 다음과 같이 계산됨.
$$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]})$$

### Arguments

dA : 현재 layer의 gradient값이 인자로 들어옵니다.<br/>
cache : forward pass에서 계산했던 linear(Z) 부분과 activation(A) 부분의 계산값들을 받음.

### Returns

dA_prev : Linear 구간의 input으로 들어왔었던, 지난 레이어의 activation을 통과한 A가 cost func에 대한 변화량.<br/>
dW : Linear 구간의 weight의 cost func에 대한 변화량.<br/>
db : Linear 구간의 bias의 cost func에 대한 변화량

$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} $$

$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}$$

$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} $$


In [35]:
def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    assert (dZ.shape == Z.shape)
    return dZ

def sigmoid_backward(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    assert (dZ.shape == Z.shape)
    return dZ

In [36]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = np.dot(dZ, cache[0].T) / m
    db = np.sum(dZ, axis=1, keepdims=True) / m
    dA_prev = np.dot(cache[1].T, dZ)

    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)

    return dA_prev, dW, db

def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db

In [37]:
def L_model_backward(AL, Y, caches):
    grads = {} # 빈 dict 호출
    L = len(caches) # layer의 갯수를 caches로 부터 받아옴.
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # Shape을 AL과 동일하게 해줌.

    # Initializing the backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1-Y, 1-AL))
    # caches index를 잡아둠.
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation="sigmoid")

    for l in reversed(range(L-1)):
        # indexing
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA"+str(l+1)], current_cache, activation="relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l+1)] = dW_temp
        grads["db" + str(l+1)] = db_temp
    return grads

In [38]:
grads = L_model_backward(AL, Y, caches)
grads

{'dA2': array([[-0.14635334,  0.11798801,  0.13252339, -0.14933579],
        [ 0.19812456, -0.15972524, -0.17940239,  0.20216203],
        [ 0.06279945, -0.05062804, -0.0568651 ,  0.06407921]]),
 'dW3': array([[0.        , 0.10525171, 0.02263592]]),
 'db3': array([[0.04261298]]),
 'dA1': array([[-0.03319042,  0.02675765,  0.        , -0.06139323],
        [ 0.07063076, -0.05694153,  0.        ,  0.07763196],
        [-0.01167404,  0.00941145,  0.        , -0.03766948],
        [ 0.07793019, -0.06282622,  0.        ,  0.05281778]]),
 'dW2': array([[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.03456726,  0.10784149, -0.0304345 ,  0.00843343],
        [ 0.        ,  0.02966002,  0.        ,  0.        ]]),
 'db2': array([[0.        ],
        [0.06014034],
        [0.0160198 ]]),
 'dA0': array([[-0.05789485,  0.03758792,  0.        , -0.0692654 ],
        [ 0.01646518, -0.00219057,  0.        ,  0.01177983],
        [ 0.01934638, -0.03058258,  0.        ,  0.02903235],

## 6. Update parameter

파라미터를 업데이트 하는 규칙은 생각보다 간편함. Learning rate인 ${α}$에 Gradient를 곱해서 현재의 param에 빼주면 새로운 param이 됨.

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} $$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} $$

### Arguments

parameters : 파라미터들이 담겨져 있는 parameter dict.<br/> grads : Gradient들이이 담겨있는는 dict.

### Returns

parameters :  업데이트 되어있는 파라미터들이 담긴 dictionary
* parameters["W" + str(l)] = ...
* parameters["b" + str(l)] = ...

In [42]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2 # layer 갯수
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
    return parameters

In [43]:
parameters = update_parameters(parameters, grads, 0.05)
parameters

{'W1': array([[-0.4887191 ,  0.36862591, -0.29428418,  0.573436  ,  0.25544048],
        [-0.88884866,  0.15291767,  0.37557805,  0.04814068,  0.72596473],
        [-0.01710925, -0.36324832, -0.1520621 ,  0.63753471,  0.93250482],
        [-0.1431176 ,  0.1759425 , -0.23868191,  0.46041071,  0.09681625]]),
 'b1': array([[ 8.04096028e-05],
        [-1.14151490e-03],
        [ 2.82824182e-05],
        [-9.74127316e-04]]),
 'W2': array([[-0.23618477, -0.49271991, -0.55171079, -0.47334635],
        [-0.16925136,  0.35110468, -0.05740103,  0.39291768],
        [-0.429569  ,  0.08531353, -0.40196401, -0.41667964]]),
 'b2': array([[ 0.        ],
        [-0.00300702],
        [-0.00080099]]),
 'W3': array([[-0.26504679,  0.35354219,  0.1125984 ]]),
 'b3': array([[-0.00213065]])}

In [41]:
parameters

{'W1': array([[-0.48801736,  0.36878669, -0.2937001 ,  0.57356559,  0.25579563],
        [-0.89222788,  0.15173943,  0.37397418,  0.04629986,  0.72582663],
        [-0.01686243, -0.36319177, -0.15185667,  0.63758029,  0.93262974],
        [-0.1446226 ,  0.17641405, -0.2385277 ,  0.46009671,  0.09729178]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.]]),
 'W2': array([[-0.23618477, -0.49271991, -0.55171079, -0.47334635],
        [-0.16752299,  0.35649675, -0.05892275,  0.39333935],
        [-0.429569  ,  0.08679653, -0.40196401, -0.41667964]]),
 'b2': array([[0.],
        [0.],
        [0.]]),
 'W3': array([[-0.26504679,  0.35880478,  0.11373019]]),
 'b3': array([[0.]])}